## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization,Activation
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [8]:

classifier=Sequential()
classifier.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(32,32,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(filters=32, kernel_size=(3,3),padding='same'))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\ProgramData\Anaconda3\envs\tf_2.1\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  del sys.path[0]
C:\ProgramData\Anaconda3\envs\tf_2.1\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  app.launch_new_instance()


Epoch 1/100
50000/50000 [==============================] - 9s 186us/step - loss: 1.3340 - accuracy: 0.5242
Epoch 2/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.9673 - accuracy: 0.6584
Epoch 3/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.8376 - accuracy: 0.7043
Epoch 4/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.7522 - accuracy: 0.7374
Epoch 5/100
50000/50000 [==============================] - 7s 132us/step - loss: 0.6880 - accuracy: 0.7589
Epoch 6/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.6329 - accuracy: 0.7786
Epoch 7/100
50000/50000 [==============================] - 7s 132us/step - loss: 0.5797 - accuracy: 0.7986
Epoch 8/100
50000/50000 [==============================] - 7s 132us/step - loss: 0.5382 - accuracy: 0.8104
Epoch 9/100
50000/50000 [==============================] - 7s 132us/step - loss: 0.4859 - accuracy: 0.8301
Epoch 10/100
50000/50000 [===========

Epoch 77/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.0441 - accuracy: 0.9850
Epoch 78/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.0276 - accuracy: 0.9909
Epoch 79/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.0310 - accuracy: 0.9894
Epoch 80/100
50000/50000 [==============================] - 6s 130us/step - loss: 0.0402 - accuracy: 0.9865
Epoch 81/100
50000/50000 [==============================] - 6s 130us/step - loss: 0.0358 - accuracy: 0.9887
Epoch 82/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.0352 - accuracy: 0.9878
Epoch 83/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.0286 - accuracy: 0.9907
Epoch 84/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.0282 - accuracy: 0.9903
Epoch 85/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.0318 - accuracy: 0.9895
Epoch 86/100
50000/50000 [==

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [9]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) # only test one picture, not x_test,y_test
classifier.predict(input_example)

array([[5.57590649e-03, 4.06227453e-04, 3.45112880e-07, 6.44863486e-01,
        3.47776890e-01, 3.19369314e-13, 1.81901180e-06, 5.34957007e-06,
        1.36993953e-03, 1.11219736e-17]], dtype=float32)

In [10]:
score = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 109us/step
Test loss: 3.870033257675171
Test accuracy: 0.6521000266075134
